In [1]:
import pandas as pd
import numpy as np   
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_csv('/media/vlad/Transcend/Microsoft VS Code/ready code/NNetworks/spam.csv',encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [10]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [13]:
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis= 1)

In [20]:
#Show the number of rows and colms
df.shape

(5572, 2)

In [21]:
#Cheking for dublicates and removing them
df.drop_duplicates(inplace= True)

In [22]:
df.shape

(5169, 2)

In [23]:
df_spam = df[df['v1'] == 'spam']

In [24]:
df_not_spam = df[df['v1'] == 'ham']

In [25]:
df.v1.value_counts()

ham     4516
spam     653
Name: v1, dtype: int64

In [26]:
df.columns

Index(['v1', 'v2'], dtype='object')

In [42]:
df_spam_or = {'ham':0,'spam':1}
df.v1 = df.v1.map(df_spam_or)

In [27]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [43]:
df

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [28]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 760 kB/s 
     |████████████████████████████████| 764 kB 3.6 MB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [44]:
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/vlad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
def process_text(v2):
    #1 Removing of punctuation
    nopunc = [char for char in v2 if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    #2 Remove stop word
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    #3 Return a list of clean words
    return clean_words 

In [46]:
df['v2'].head().apply(process_text)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: v2, dtype: object

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer= process_text).fit_transform(df['v2'])

In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(message_bow,df['v1'],test_size = 0.2,random_state = 0)

In [49]:
message_bow.shape

(5169, 11304)

In [50]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train,y_train)

MultinomialNB()

In [51]:
print(classifier.predict(X_train))

[0 0 0 ... 0 0 0]


In [52]:
print(y_train.values)

[0 0 0 ... 0 0 0]


In [55]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train,pred))
print('Confiusion matrix: \n' ,confusion_matrix(y_train,pred))
print()
print('Accuracy:', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3631
           1       0.98      0.98      0.98       504

    accuracy                           1.00      4135
   macro avg       0.99      0.99      0.99      4135
weighted avg       1.00      1.00      1.00      4135

Confiusion matrix: 
 [[3623    8]
 [  11  493]]

Accuracy: 0.9954050785973397


In [56]:
print('Predicted value:', classifier.predict(X_test))
print('Actual value:', y_test.values)

Predicted value: [0 0 0 ... 0 0 0]
Actual value: [0 0 0 ... 0 0 0]


In [57]:
pred = classifier.predict(X_test)
print(classification_report(y_test,pred))
print('Confiusion matrix: \n' ,confusion_matrix(y_test,pred))
print()
print('Accuracy:', accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       885
           1       0.80      0.93      0.86       149

    accuracy                           0.96      1034
   macro avg       0.89      0.94      0.92      1034
weighted avg       0.96      0.96      0.96      1034

Confiusion matrix: 
 [[850  35]
 [ 11 138]]

Accuracy: 0.9555125725338491
